# Importing of All Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from sklearn.inspection import permutation_importance
from tqdm import tqdm

# Scenario: Creating a Set of Machine Learning Friendly Features from EHR Data to Predict Diabetes Onset

First we will load in the necessary data files

In [2]:
def load_data_for_file(filename):
    print(f"Loading data for {filename}")
    df = pd.concat([ # use pd.concat to append/concatenate the data for all states together into a single frame
        pd.read_parquet(f"https://dicbworkshops.s3.amazonaws.com/{output_dir}/parquet/{filename}") # use read_csv to load the data from each output directory
        for output_dir in tqdm(['output_hi', 'output_ma', 'output_tx', 'output_wa'], leave=True, position=0) # loop over each output directory
    ])
    return df

In [3]:
# load in the conditions
conditions = load_data_for_file('conditions.parquet')
# load in the observations
observations = load_data_for_file('observations.parquet')
# load in the medications
medications = load_data_for_file('medications.parquet')
# load in the procedures
procedures = load_data_for_file('procedures.parquet')
# load in the patients
patients = load_data_for_file('patients.parquet')

Loading data for conditions.parquet


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.44it/s]


Loading data for observations.parquet


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.15s/it]


Loading data for medications.parquet


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.27it/s]


Loading data for procedures.parquet


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.01s/it]


Loading data for patients.parquet


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.04it/s]


# Definition of Helper Functions Used to Compose the Larger Pipeline

In [4]:
# function to construct a lookup table of condition onset dates for patients based on the conditions table and a provided code
def get_patient_onset_dates(conditions, code):
    patients_with_condition = (
        conditions.query('CODE == @code') # get all patients diagnosed with the code
        .sort_values(by=['PATIENT', 'START']) # sort the data by patient ID and then start date
        .drop_duplicates(subset=['PATIENT', 'START'], keep='first') # drop duplicates, keeping the instance with the earliest start date
    )

    # now build a lookup table/dictionary to map each patient's ID to the date of their earliest onset
    patient_onset_dates = {
        row['PATIENT']: row['START']
        for _, row in patients_with_condition.iterrows()
    }
    return patient_onset_dates

In [5]:
# function to construct simplified date columns for the observations, medications, and procedures tables
def get_simplified_data(df, date_col, simplified_col='DATE_SIMPLE'):
    return df.assign(**{
        simplified_col: lambda x: pd.to_datetime(x[date_col]).dt.date.astype('str')
    })

In [6]:
# function to filter out post diagnosis records from a table based on patient onset dates
def filter_data_by_onset_dates(df, patient_onset_dates, date_column='DATE_SIMPLE'):
    data_filtered = []
    for _, row in tqdm(df.iterrows(), total=len(df), position=0, leave=True):
        patient = row['PATIENT']
        date = row[date_column]
        if patient in patient_onset_dates and patient_onset_dates[patient] > date:
            data_filtered.append(row)
    return pd.DataFrame(data_filtered)

In [7]:
# function to unify the records for the four different types of events/ecounters into a single table
def get_unified_records(conditions, observations, medications, procedures):
    return pd.concat([
        conditions[['PATIENT', 'START', 'CODE', 'DESCRIPTION']].assign(
            EVENT_TYPE='CONDITION',
        ).rename(columns={'START': 'DATE'}),
        observations[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
            EVENT_TYPE='OBSERVATION',
        ).rename(columns={'DATE_SIMPLE': 'DATE'}),
        medications[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
            EVENT_TYPE='MEDICATION',
        ).rename(columns={'DATE_SIMPLE': 'DATE'}),
        procedures[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
            EVENT_TYPE='PROCEDURE',
        ).rename(columns={'DATE_SIMPLE': 'DATE'})
])

In [8]:
# function to get condensed record data from unified records
def get_condensed_record_data(unified_records):
    # condense the records into a pipe-delimited string of event tokens per patient, 
    # where each token is of the form <EVENT_TYPE>::<CODE>
    records_condensed = unified_records.assign(
        EVENT_TOKEN=lambda x: x['EVENT_TYPE'] + '::' + x['CODE'].astype(str) + '|'
    ).groupby(['PATIENT'])['EVENT_TOKEN'].sum().reset_index()
    return records_condensed

In [9]:
# function to vectorize the unified record data into binary occurence format
def get_multihot_vector_representation(condensed_records, vectorizer, train=True):
    # now get the multi-hot representation from the vectorizer
    if train:
        # if this is the training set, fit before transforming
        return vectorizer.fit_transform(condensed_records['EVENT_TOKEN'])
    else:
        #otherwise, just transform
        return vectorizer.transform(condensed_records['EVENT_TOKEN'])
    

In [10]:
# function to add patient ages to a given dataframe, and compute age bins from those ages
def get_aged_patient_data(events_df, patients_df, bin_width=5, date_col='DATE_SIMPLE'):
    # first merge in the birthdates from the patients dataframe
    merged = events_df.merge(
        patients_df[['Id', 'BIRTHDATE']],
        left_on='PATIENT',
        right_on='Id'
    )
    # now calculate the age from the birthdate and the date column
    aged = merged.assign(
        AGE=lambda x: (pd.to_datetime(x[date_col]) - pd.to_datetime(x['BIRTHDATE'])).dt.days // 365
    )
    # now use the calculated age to compute age bins using pd.cut
    age_binned = aged.assign(
        AGE_BIN=lambda x: pd.cut(x['AGE'], bins=list(np.arange(0, x['AGE'].max() + bin_width, bin_width)), include_lowest=True)
    )
    # now get the human readable age bin
    result = age_binned.assign(
        AGE_RANGE=lambda x: x['AGE_BIN'].apply(lambda b: f"{int(b.left)}-{int(b.right)}" if b.left <= 0 else f"{int(b.left)+1}-{int(b.right)}")
    )
    # return the age_binned data
    return result

In [11]:
# function to build age_binned distributions for numeric observations
def get_age_binned_distributions(observations_df):
    # first make sure that we are only considering numeric observations
    numeric_obs = observations_df.query('TYPE == "numeric"')
    # build the datastructure to store the distributions
    observation_distributions = {
        code: {}
        for code in numeric_obs['CODE'].unique()
    }
    # now iterate over the observations, and construct the age binned distributions
    for _, row in tqdm(numeric_obs.iterrows(), total=len(numeric_obs)):
        # get the value, code, and age_range
        value = row['VALUE']
        code = row['CODE']
        age_range = row['AGE_RANGE']
        # add the value to the distribution for the corresponding code and age range
        observation_distributions[code][age_range] = observation_distributions[code].get(age_range, []) + [value]
    # now sort all of the distributions
    for age_bin_dists in observation_distributions.values():
        for age_bin_dist in age_bin_dists.values():
            age_bin_dist.sort()
    return observation_distributions
        

In [12]:
# Fast implementation of percentile ranking, assume group is sorted
def fast_percentile_ranking(group, value):
    if len(group) == 0:
        return np.nan
    return recursive_ranking(group, value, 0, len(group) - 1)

def recursive_ranking(group, value, start, end):
    if start > end:
        return start / len(group) * 100
    mid = (start + end) // 2
    if value > group[mid]:
        return recursive_ranking(group, value, mid+1, end)
    elif value < group[mid]:
        return recursive_ranking(group, value, start, mid-1)
    elif len(group) % 2 == 0:
        high = (mid+1) / len(group) * 100
        return high
    else:
        low = mid / len(group) * 100
        high = (mid+1) / len(group) * 100
        return (low + high) / 2

In [13]:
# function to compute the percentile score of all numeric observations against those in their age cohort
def get_percentile_ranked_observations(observations_df, observation_distributions):
    observations_ranked = observations_df.assign(
        PERCENTILE_RANK=lambda x: x.apply(
            lambda row: fast_percentile_ranking(observation_distributions.get(row['CODE'], {}).get(row['AGE_RANGE'], []), row['VALUE']),
            axis=1
        )
    )
    return observations_ranked

In [14]:
# function to add the label for the percentile score of the ranked numeric observations to the CODE column
def get_percentile_labeled_observations(observations_df, percentiles=4):
    # use the pd.cut function to bin the PERCENTILE_RANK into percentiles many bins
    bin_width = 100 // percentiles
    observations_df['PERCENTILE'] = pd.cut(
        observations_df['PERCENTILE_RANK'], 
        bins=np.arange(0, 100+bin_width, bin_width), 
        include_lowest=True
    )
    observations_df['CODE'] = (
        observations_df['CODE'] + '__' + 
        observations_df['PERCENTILE'].map(lambda interval: interval.left).astype('int').astype('str') + "th - " +
        observations_df['PERCENTILE'].map(lambda interval: interval.right).astype('int').astype('str') +
        'th Percentile'
    )
    return observations_df
    

In [15]:
def train_and_evaluate_classifier_kfold(clf, X, y, k=5):
    metrics = []
    kfold = StratifiedKFold(n_splits=k, random_state=913, shuffle=True)
    for i, (train_index, test_index) in tqdm(enumerate(kfold.split(X, y)), total=k, position=0, leave=True):
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        # fit the model on the training fold
        clf.fit(train_x, train_y)
        # evaluate the model on the validation fold
        preds = clf.predict(test_x)
        scores = clf.predict_proba(test_x)[:, 1]
        # get the AUROC
        fpr, tpr, _ = roc_curve(test_y, scores)
        auroc = auc(fpr, tpr)
        # get the confusion matrix
        cm = confusion_matrix(test_y, preds)
        # save the metrics
        metrics.append({
            'AUROC': auroc,
            'Precision': cm[1, 1] / cm[:, 1].sum(),
            'Recall': cm[1, 1] / cm[1].sum(),
            'Specificity': cm[0, 0] / cm[0].sum()
        })
    return pd.DataFrame(metrics)

In [16]:
def get_feature_importance_rankings(clf, vectorizer, all_records):
    reverse_lookup = {
        value: key for key, value in vectorizer.vocabulary_.items()
    }
    top_50_indices = np.argsort(clf.feature_importances_)[-50:]
    top50_features = [reverse_lookup[idx] for idx in top_50_indices]
    top50_importances = clf.feature_importances_[top_50_indices]
    top_features_df = pd.DataFrame({
        'FEATURE_NAME': top50_features,
        'FEATURE_IMPORTANCE': top50_importances
    }).assign(
        CODE=lambda x: x['FEATURE_NAME'].str.split('::').apply(lambda pair: pair[1])
    ).merge(
        all_records[['CODE', 'DESCRIPTION']].drop_duplicates().astype({'CODE': str}),
        on='CODE',
    ).sort_values(by='FEATURE_IMPORTANCE', ascending=False)
    return top_features_df

In [17]:
def get_feature_importance_rankings_lr(clf, vectorizer, all_records):
    reverse_lookup = {
        value: key for key, value in vectorizer.vocabulary_.items()
    }
    top_50_indices = np.argsort(np.abs(clf.coef_[0]))[-50:]
    top50_features = [reverse_lookup[idx] for idx in top_50_indices]
    top50_importances = clf.coef_[0][top_50_indices]
    top_features_df = pd.DataFrame({
        'FEATURE_NAME': top50_features,
        'FEATURE_IMPORTANCE': top50_importances,
        'ABSOLUTE_IMPORTANCE': np.abs(top50_importances)
    }).assign(
        CODE=lambda x: x['FEATURE_NAME'].str.split('::').apply(lambda pair: pair[1])
    ).merge(
        all_records[['CODE', 'DESCRIPTION']].drop_duplicates().astype({'CODE': str}),
        on='CODE',
    ).sort_values(by='ABSOLUTE_IMPORTANCE', ascending=False)
    return top_features_df

## Pipeline Specifications
For this exercise, we are interested in studying patients with a diagnosis of Type-2 diabetes \
We select these from the conditions table based on the SNOMED code `44054006`

In [18]:
type2_code = 44054006

## Option 1: Bag of Labeled Clinical Encounters (Many-hot/multi-hot encoding)
The simplest feature representation we can create and test is a binary vector (many-hot/multi-hot) representation \
which encodes the occurence or lack-therof of different clinical encounters/event in each patient's EHR record \
To construct this representation, we can use the scikit-learn package's `CountVectorizer` class

In [19]:
def pipeline_option1():
    print("Getting onset dates...")
    # get onset dates of patients for type-2 diabetes with SNOMED code 44054006
    type2_onset_dates = get_patient_onset_dates(conditions, type2_code)

    print("Simplifying dates...")
    # add simplified date columns to the observations, medications, and procedures
    observations_simple = get_simplified_data(observations, 'DATE')
    medications_simple = get_simplified_data(medications, 'START')
    procedures_simple = get_simplified_data(procedures, 'START')

    # drop all cause of death observations from the data
    observations_non_cod = observations_simple[observations_simple['CODE'] != '69453-9']

    print("Filtering out postdiagnosis events...")
    # now we will get filtered data for the type 2 patients to exclude post-diagnosis information
    conditions_filtered = filter_data_by_onset_dates(conditions, type2_onset_dates, 'START')
    observations_filtered = filter_data_by_onset_dates(observations_non_cod, type2_onset_dates)
    medications_filtered = filter_data_by_onset_dates(medications_simple, type2_onset_dates)
    procedures_filtered = filter_data_by_onset_dates(procedures_simple, type2_onset_dates)

    # now we will save the set of unique type2 patients who have a pre-diagnosis record
    type2_patients = pd.concat([
        conditions_filtered['PATIENT'],
        observations_filtered['PATIENT'],
        medications_filtered['PATIENT'],
        procedures_filtered['PATIENT']
    ]).unique()

    # now extract the observations for the non-type2 patients
    conditions_non_type2 = conditions[~conditions['PATIENT'].isin(type2_patients)]
    observations_non_type2 = observations_non_cod[~observations_non_cod['PATIENT'].isin(type2_patients)]
    medications_non_type2 = medications_simple[~medications_simple['PATIENT'].isin(type2_patients)]
    procedures_non_type2 = procedures_simple[~procedures_simple['PATIENT'].isin(type2_patients)]

    
    print("Unifying and condensing records...")
    # now we will unify the data together into a single set of records
    unified_records_type2 = get_unified_records(
        conditions_filtered,
        observations_filtered,
        medications_filtered,
        procedures_filtered
    )

    unified_records_non_type2 = get_unified_records(
        conditions_non_type2,
        observations_non_type2,
        medications_non_type2,
        procedures_non_type2
    )
    
    # now we will condense the records for the type 2 and non-type 2 patients
    type2_condensed = get_condensed_record_data(unified_records_type2)
    non_type2_condensed = get_condensed_record_data(unified_records_non_type2)

    # now we will concatenate the two datasets together and label them
    all_data_condensed = pd.concat([
        type2_condensed.assign(LABEL=1),
        non_type2_condensed.assign(LABEL=0)
    ])

    print("Vectorizing data...")
    # now we will get the multi-hot vector representation for the records
    vectorizer = CountVectorizer(
        binary=True,
        tokenizer=lambda x: x.split('|'),
        token_pattern=None,
        lowercase=False
    )

    # now vectorize the data to get a multi-hot representation
    multi_hot_vectors = get_multihot_vector_representation(all_data_condensed, vectorizer)

    print("Fitting and evaluating classifier...")
    # now we will construct the random forest classifier and evaluate it using KFold cross-validation
    clf = LogisticRegression(
        solver='lbfgs',
        max_iter=1000,
        random_state=913
    )

    # now we train and evaluate the classifier
    results = train_and_evaluate_classifier_kfold(
        clf, multi_hot_vectors, all_data_condensed['LABEL'].to_numpy()
    )

    # now return the results, trained classifier, and vectorizer
    return results, clf, vectorizer, pd.concat([unified_records_type2, unified_records_non_type2])

    
    

In [20]:
results, clf, vectorizer, all_records = pipeline_option1()

Getting onset dates...
Simplifying dates...
Filtering out postdiagnosis events...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 839891/839891 [00:11<00:00, 73948.06it/s]


Unifying and condensing records...
Vectorizing data...
Fitting and evaluating classifier...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 15.90it/s]


In [21]:
results

,AUROC,Precision,Recall,Specificity
0,0.955939,0.955556,0.767857,0.997674
1,0.961483,0.936170,0.785714,0.996512
2,0.971200,0.958333,0.821429,0.997674
3,0.933451,0.933333,0.750000,0.996512
4,0.971083,0.918367,0.803571,0.995343


In [22]:
results['AUROC'].mean()

np.float64(0.9586311489834042)

### The model appears to be performing decently so far, but is that where the story ends, or does the plot thicken?
One thing that we can do is audit the way our current model is behaving, and what its predictions are based \
on, by looking at feature importance rankings. Here we compute the permutation importance of the model features \
on a validation set, and focus in on the top 20 features ranked by importance.

In [23]:
top_features_df = get_feature_importance_rankings_lr(clf, vectorizer, all_records)

In [24]:
top_features_df

,FEATURE_NAME,FEATURE_IMPORTANCE,ABSOLUTE_IMPORTANCE,CODE,DESCRIPTION
52,MEDICATION::1649987,1.251726,1.251726,1649987,doxycycline hyclate 100 MG
51,PROCEDURE::1263416007,-1.035929,1.035929,1263416007,Removal of intrauterine contraceptive device (...
50,MEDICATION::617311,1.012106,1.012106,617311,atorvastatin 40 MG Oral Tablet
49,CONDITION::714628002,0.948952,0.948952,714628002,Prediabetes (finding)
48,CONDITION::423315002,-0.907909,0.907909,423315002,Limited social contact (finding)
47,OBSERVATION::21000-5,-0.907855,0.907855,21000-5,Erythrocyte distribution width [Entitic volume...
46,OBSERVATION::32207-3,-0.907855,0.907855,32207-3,Platelet distribution width [Entitic volume] i...
45,OBSERVATION::32623-1,-0.907855,0.907855,32623-1,Platelet mean volume [Entitic volume] in Blood...
44,CONDITION::162864005,0.900334,0.900334,162864005,Body mass index 30+ - obesity (finding)
43,OBSERVATION::59576-9,-0.883786,0.883786,59576-9,Body mass index (BMI) [Percentile] Per age and...


## Option 2: Binary Occurrence with Inclusion of Discretized Numeric Features
While the feature importance rankings revealed known co-morbidities (e.g., tooth loss) and known risk-factors for diabetes (e.g., Prediabetes, BMI) \
we have left out entirely the numeric data from the observations (lab and vital sign measures) which are likely to contain important information. \
Here we look at one technique for incorporating this information while maintaining the binary occurence vector representation that we used previously

In [27]:
def pipeline_option2():
    print("Getting onset dates...")
    # get onset dates of patients for type-2 diabetes with SNOMED code 44054006
    type2_onset_dates = get_patient_onset_dates(conditions, type2_code)

    print("Simplifying dates...")
    # add simplified date columns to the observations, medications, and procedures
    observations_simple = get_simplified_data(observations, 'DATE')
    medications_simple = get_simplified_data(medications, 'START')
    procedures_simple = get_simplified_data(procedures, 'START')

    # drop all cause of death observations from the data
    observations_non_cod = observations_simple[observations_simple['CODE'] != '69453-9']
    

    print("Filtering out postdiagnosis events...")
    # now we will get filtered data for the type 2 patients to exclude post-diagnosis information
    conditions_filtered = filter_data_by_onset_dates(conditions, type2_onset_dates, 'START')
    observations_filtered = filter_data_by_onset_dates(observations_non_cod, type2_onset_dates)
    medications_filtered = filter_data_by_onset_dates(medications_simple, type2_onset_dates)
    procedures_filtered = filter_data_by_onset_dates(procedures_simple, type2_onset_dates)

    # now we will save the set of unique type2 patients who have a pre-diagnosis record
    type2_patients = pd.concat([
        conditions_filtered['PATIENT'],
        observations_filtered['PATIENT'],
        medications_filtered['PATIENT'],
        procedures_filtered['PATIENT']
    ]).unique()

    # now extract the observations for the non-type2 patients
    conditions_non_type2 = conditions[~conditions['PATIENT'].isin(type2_patients)]
    observations_non_type2 = observations_non_cod[~observations_non_cod['PATIENT'].isin(type2_patients)]
    medications_non_type2 = medications_simple[~medications_simple['PATIENT'].isin(type2_patients)]
    procedures_non_type2 = procedures_simple[~procedures_simple['PATIENT'].isin(type2_patients)]

    print("Adding patient ages to observations...")
    ###### NEW STEP: Add ages to the observations data and compute age bins ######
    observations_type2_aged = get_aged_patient_data(observations_filtered, patients)
    observations_non_type2_aged = get_aged_patient_data(observations_non_type2, patients)

    observations_type2_numeric = observations_type2_aged.query('TYPE == "numeric"')
    observations_non_type2_numeric = observations_non_type2_aged.query('TYPE == "numeric"')

    print("Computing age-binned numeric observation distributions...")
    ###### NEW STEP: Compute the age-binned numeric observation distributions ######
    observation_distributions = get_age_binned_distributions(
        pd.concat([observations_type2_numeric, observations_non_type2_numeric])
    )

    print("Computing percentile rank of numeric observations...")
    ###### NEW STEP: Compute the percentile rank of each observation against those for patients in the same age cohort ######
    observations_type2_ranked = get_percentile_ranked_observations(observations_type2_numeric, observation_distributions)
    observations_non_type2_ranked = get_percentile_ranked_observations(observations_non_type2_numeric, observation_distributions)

    get_percentile_labeled_observations(observations_type2_ranked)
    get_percentile_labeled_observations(observations_non_type2_ranked)

    ###### Make sure we recombine the numeric and non-numeric observations ######
    observations_type2_final = pd.concat([
        observations_type2_ranked.drop(columns=['PERCENTILE', 'PERCENTILE_RANK']),
        observations_type2_aged.query('TYPE != "numeric"')
    ])

    observations_non_type2_final = pd.concat([
        observations_non_type2_ranked.drop(columns=['PERCENTILE', 'PERCENTILE_RANK']),
        observations_non_type2_aged.query('TYPE != "numeric"')
    ])

    print("Unifying and condensing records...")
    # now we will unify the data together into a single set of records
    unified_records_type2 = get_unified_records(
        conditions_filtered,
        observations_type2_final,
        medications_filtered,
        procedures_filtered
    )

    unified_records_non_type2 = get_unified_records(
        conditions_non_type2,
        observations_non_type2_final,
        medications_non_type2,
        procedures_non_type2
    )

    # now we will condense the records for the type 2 and non-type 2 patients
    type2_condensed = get_condensed_record_data(unified_records_type2)
    non_type2_condensed = get_condensed_record_data(unified_records_non_type2)

    # now we will concatenate the two datasets together and label them
    all_data_condensed = pd.concat([
        type2_condensed.assign(LABEL=1),
        non_type2_condensed.assign(LABEL=0)
    ])

    print("Vectorizing data...")
    # now we will get the multi-hot vector representation for the records
    vectorizer = CountVectorizer(
        binary=True,
        tokenizer=lambda x: x.split('|'),
        token_pattern=None,
        lowercase=False
    )

    # now vectorize the data to get a multi-hot representation
    multi_hot_vectors = get_multihot_vector_representation(all_data_condensed, vectorizer)

    print("Fitting and evaluating classifier...")
    # now we will construct the random forest classifier and evaluate it using KFold cross-validation
    clf = LogisticRegression(
        solver='lbfgs',
        max_iter=1000,
        random_state=913
    )

    # now we train and evaluate the classifier
    results = train_and_evaluate_classifier_kfold(
        clf, multi_hot_vectors, all_data_condensed['LABEL'].to_numpy()
    )

    # now return the results, trained classifier, and vectorizer
    return results, clf, vectorizer, pd.concat([unified_records_type2, unified_records_non_type2])

In [28]:
results_p2, clf_p2, vectorizer_p2, all_records_p2 = pipeline_option2()

Getting onset dates...
Simplifying dates...
Filtering out postdiagnosis events...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 839891/839891 [00:11<00:00, 73130.01it/s]


Adding patient ages to observations...
Computing age-binned numeric observation distributions...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2125494/2125494 [00:39<00:00, 53145.07it/s]


Computing percentile rank of numeric observations...
Unifying and condensing records...
Vectorizing data...
Fitting and evaluating classifier...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 13.53it/s]


In [29]:
results_p2

,AUROC,Precision,Recall,Specificity
0,0.978841,0.978261,0.803571,0.998837
1,0.981250,0.979167,0.839286,0.998837
2,0.986316,0.942308,0.875000,0.996512
3,0.970037,1.000000,0.803571,1.000000
4,0.990749,0.979167,0.839286,0.998836


In [30]:
results_p2['AUROC'].mean()

np.float64(0.981438878553831)

In [31]:
top_features_p2 = get_feature_importance_rankings_lr(clf_p2, vectorizer_p2, all_records_p2)

In [32]:
top_features_p2

,FEATURE_NAME,FEATURE_IMPORTANCE,ABSOLUTE_IMPORTANCE,CODE,DESCRIPTION
53,OBSERVATION::QALY__0th - 25th Percentile,-1.533749,1.533749,QALY__0th - 25th Percentile,QALY
52,OBSERVATION::21000-5__75th - 100th Percentile,-0.944550,0.944550,21000-5__75th - 100th Percentile,Erythrocyte distribution width [Entitic volume...
51,OBSERVATION::32623-1__25th - 50th Percentile,-0.905819,0.905819,32623-1__25th - 50th Percentile,Platelet mean volume [Entitic volume] in Blood...
50,CONDITION::162864005,0.861879,0.861879,162864005,Body mass index 30+ - obesity (finding)
49,OBSERVATION::18262-6__75th - 100th Percentile,-0.857407,0.857407,18262-6__75th - 100th Percentile,Low Density Lipoprotein Cholesterol
48,OBSERVATION::70274-6__0th - 25th Percentile,-0.801240,0.801240,70274-6__0th - 25th Percentile,Generalized anxiety disorder 7 item (GAD-7) to...
47,OBSERVATION::718-7__50th - 75th Percentile,-0.768854,0.768854,718-7__50th - 75th Percentile,Hemoglobin
46,OBSERVATION::718-7__50th - 75th Percentile,-0.768854,0.768854,718-7__50th - 75th Percentile,Hemoglobin [Mass/volume] in Blood
45,OBSERVATION::789-8__0th - 25th Percentile,-0.767156,0.767156,789-8__0th - 25th Percentile,Erythrocytes [#/volume] in Blood by Automated ...
44,OBSERVATION::72514-3__75th - 100th Percentile,-0.757087,0.757087,72514-3__75th - 100th Percentile,Pain severity - 0-10 verbal numeric rating [Sc...


## Option 2: Bag of Character N-Grams
One way that we can potentially improve on our earlier feature representation is to use a bag of N-grams \
which encodes the occurence or lack-therof of different substrings from clinical encounter descriptions \
To construct this representation, we can again make use of the scikit-learn package's `CountVectorizer` class \
but instead of using the default `"word"` analyzer, we will instead use the `"char_wb"` analyzer

In [40]:
vectorizer_ngram = CountVectorizer(
    binary=True,
    lowercase=True,
    ngram_range=(5, 5),
    analyzer='char_wb'
)

In [41]:
# condense the records into a concatenated description per patient, 
# wherein we take all event descriptions and combine them together into a single
# description that encompasses all of the clinical events for that patient
train_records_condensed = train_records.assign(
    DESCRIPTION_WS=lambda x: x['DESCRIPTION'] + ' '
).groupby('PATIENT')['DESCRIPTION_WS'].sum().reset_index()

In [42]:
train_records_condensed.iloc[0]['DESCRIPTION_WS']

'Medication review due (situation) Medication review due (situation) Medication review due (situation) Medication review due (situation) Medication review due (situation) Otitis media (disorder) Medication review due (situation) Medication review due (situation) Laceration - injury (disorder) Facial laceration (disorder) Medication review due (situation) Viral sinusitis (disorder) Childhood asthma (disorder) Medication review due (situation) Sprain (morphologic abnormality) Sprain of ankle (disorder) Medication review due (situation) Body Height Pain severity - 0-10 verbal numeric rating [Score] - Reported Body Weight Weight-for-length Per age and sex Head Occipital-frontal circumference Percentile Head Occipital-frontal circumference Diastolic Blood Pressure Systolic Blood Pressure Heart rate Respiratory rate Leukocytes [#/volume] in Blood by Automated count Erythrocytes [#/volume] in Blood by Automated count Hemoglobin [Mass/volume] in Blood Hematocrit [Volume Fraction] of Blood by A

In [43]:
train_data_final = train_records_condensed.assign(
    LABEL=lambda x: x['PATIENT'].isin(type2_patients).astype(int)
)

### Now that we have a condensed representation, we will vectorize

In [44]:
ngram_vectors = vectorizer_ngram.fit_transform(train_data_final['DESCRIPTION_WS'])

### Now We Will Test the performance of this representation on the Training Set Using KFold Cross Validation

In [45]:
kfold = StratifiedKFold(n_splits=5)
clf = RandomForestClassifier()

metrics = []
for i, (train_index, test_index) in tqdm(enumerate(kfold.split(ngram_vectors, train_data_final['LABEL'])), total=5, position=0, leave=True):
    train_x, train_y = ngram_vectors[train_index], train_data_final['LABEL'].to_numpy()[train_index]
    test_x, test_y = ngram_vectors[test_index], train_data_final['LABEL'].to_numpy()[test_index]
    # fit the model on the training fold
    clf.fit(train_x, train_y)
    # evaluate the model on the validation fold
    preds = clf.predict(test_x)
    scores = clf.predict_proba(test_x)[:, 1]
    # get the AUROC
    fpr, tpr, _ = roc_curve(test_y, scores)
    auroc = auc(fpr, tpr)
    # get the confusion matrix
    cm = confusion_matrix(test_y, preds)
    # save the metrics
    metrics.append({
        'AUROC': auroc,
        'Precision': cm[1, 1] / cm[:, 1].sum(),
        'Recall': cm[1, 1] / cm[1].sum(),
        'Specificity': cm[0, 0] / cm[0].sum()
    })
pd.DataFrame(metrics)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.37it/s]


,AUROC,Precision,Recall,Specificity
0,0.949739,1.0,0.795455,1.0
1,0.900388,1.0,0.688889,1.0
2,0.952515,1.0,0.777778,1.0
3,0.956930,1.0,0.822222,1.0
4,0.950752,1.0,0.711111,1.0


### Let's Again Look at the Top 20 Features Ranked by Importance

In [46]:
# split our training dataset into "train" and "test" sets to fit the model for explainability purposes
train_x, test_x, train_y, test_y = train_test_split(ngram_vectors, train_data_final['LABEL'].to_numpy(), test_size=0.2, stratify=train_data_final['LABEL'])

In [47]:
clf.fit(train_x, train_y)

RandomForestClassifier()

In [48]:
print(classification_report(test_y, clf.predict(test_x)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       687
           1       1.00      0.82      0.90        45

    accuracy                           0.99       732
   macro avg       0.99      0.91      0.95       732
weighted avg       0.99      0.99      0.99       732



In [53]:
r = permutation_importance(clf, np.asarray(train_x.todense()), train_y, n_repeats=2, n_jobs=-1, scoring='recall')

In [54]:
importances_df = pd.DataFrame({
    'Feature': vectorizer_ngram.get_feature_names_out(),
    'mean_importance': r.importances_mean
})

In [55]:
top20 = importances_df.sort_values(by='mean_importance', ascending=False).head(20)

In [56]:
top20

,Feature,mean_importance
5423,oriny,0.005587
2959,ence,0.005587
1324,vict,0.005587
4034,iral,0.005587
7116,ture,0.005587
3979,inusi,0.005587
2395,colon,0.002793
6756,table,0.002793
2327,cilia,0.002793
1210,tabl,0.002793


In [ ]:
for _, row in tqdm(top20.iterrows(), total=20, position=0, leave=True):
    
    